# Настройки

In [ ]:
import torch

# Пути
MODEL_NAME = "guidobenb/DarkBERT-finetuned-ner"  # Выбранная модель
DATASET_PATH = "./datasets"  # Папка с локальными датасетами
OUTPUT_DIR = "./trained_model"  # Папка для сохранения модели
LOG_FILE = "./training.log"  # Файл логов

# Параметры обучения
NUM_EPOCHS = 3
BATCH_SIZE = 8
LEARNING_RATE = 5e-5

# Параметры железа
USE_GPU = torch.cuda.is_available()
DEVICE = "cuda" if USE_GPU else "cpu"
NUM_WORKERS = 4  # Количество потоков загрузки данных
RAM_LIMIT = "32GB"  # Ограничение ОЗУ (необязательно)

# Загрузка данных

In [ ]:
import os
import json
from datasets import load_dataset


def load_data(source="huggingface", dataset_name="kaykyramos/onlyfans-preferences"):
    """
    Загружает данные для обучения.
    :param source: "huggingface" или "local"
    :param dataset_name: название датасета (для Hugging Face) или путь к локальному файлу
    """
    if source == "huggingface":
        print(f"Загружаем датасет {dataset_name} с Hugging Face...")
        dataset = load_dataset(dataset_name)
    else:
        print(f"Загружаем локальный датасет из {dataset_name}...")
        if not os.path.exists(dataset_name):
            raise FileNotFoundError(f"Файл {dataset_name} не найден!")
        with open(dataset_name, "r", encoding="utf-8") as f:
            data = json.load(f)
        dataset = {"train": data}  # Преобразуем в формат, похожий на Hugging Face

    print("Данные загружены!")
    return dataset

# Логирование и проверка повторного обучения

In [ ]:
import hashlib


def get_data_hash(dataset):
    """
    Создает хеш от данных, чтобы проверить, не обучалась ли модель на них ранее.
    """
    data_str = json.dumps(dataset, sort_keys=True)
    return hashlib.md5(data_str.encode()).hexdigest()


def check_if_trained(dataset_hash):
    """
    Проверяет, обучалась ли модель на этих данных.
    """
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, "r") as f:
            trained_hashes = f.read().splitlines()
        return dataset_hash in trained_hashes
    return False


def log_training(dataset_hash):
    """
    Записывает хеш датасета в лог, чтобы избежать повторного обучения.
    """
    with open(LOG_FILE, "a") as f:
        f.write(dataset_hash + "\n")

# Обучение модели

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, TrainingArguments, Trainer


def train_model(dataset):
    """
    Обучает модель на указанном датасете.
    """
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME).to(DEVICE)

    # Токенизация данных
    tokenized_datasets = dataset.map(lambda x: tokenizer(x["text"], truncation=True, padding="max_length"), batched=True)

    # Проверяем, обучалась ли модель на этом датасете
    dataset_hash = get_data_hash(tokenized_datasets["train"])
    if check_if_trained(dataset_hash):
        print("⚠️ Эта модель уже обучалась на данных. Пропускаем обучение.")
        return

    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        logging_dir="./logs",
        logging_steps=500,
        save_total_limit=2,
        fp16=USE_GPU,
        dataloader_num_workers=NUM_WORKERS
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets.get("test", None),
        tokenizer=tokenizer
    )

    trainer.train()
    trainer.save_model(OUTPUT_DIR)
    log_training(dataset_hash)
    print(f"✅ Модель сохранена в {OUTPUT_DIR}")

# Интерактивная CLI-панель

In [ ]:
import argparse

def main():
    parser = argparse.ArgumentParser(description="Интерактивная настройка обучения DarkBERT")
    parser.add_argument("--source", choices=["huggingface", "local"], default="huggingface", help="Источник данных")
    parser.add_argument("--dataset", type=str, default="kaykyramos/onlyfans-preferences", help="Название датасета или путь")
    parser.add_argument("--epochs", type=int, default=3, help="Количество эпох")
    parser.add_argument("--batch_size", type=int, default=8, help="Размер батча")
    parser.add_argument("--lr", type=float, default=5e-5, help="Скорость обучения")

    args = parser.parse_args()

    # Загружаем данные
    dataset = load_data(args.source, args.dataset)

    # Обучаем модель
    train_model(dataset)

if __name__ == "__main__":
    main()